In [0]:
# Downloading and parsing the initial text file

import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt', 
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()

print('Corpus length : ', len(text))

Corpus length :  600893


In [0]:
# Vectorizing sequences of characters

# extract sequences of 60 characters
maxlen = 60
#sample a new sequence every three characters
step = 3

sentences = []
next_chars = []

for i in range(0, len(text)-maxlen, step):
  sentences.append(text[i:i+maxlen])
  next_chars.append(text[i+maxlen])

print('Number of sentences : ', len(sentences))

chars = sorted(list(set(text)))

print('Unique characters : ', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(next_chars), len(chars)), dtype=np.bool)

for i, sentence in  enumerate(sentences):
  for j, char in enumerate(sentence):
    x[i, j, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

Number of sentences :  200278
Unique characters :  57
Vectorization...


In [0]:
# Single-layer LSTM model for next-character prediction

from keras.layers import Dense, LSTM

model = keras.models.Sequential()

model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(optimizer=keras.optimizers.RMSprop(lr=0.01),
              loss='categorical_crossentropy')

In [0]:
# Function to sample the next character given the model’s predictions

def sample(preds, temperature):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  preds = np.random.multinomial(1, preds, 1)
  
  return np.argmax(preds)

In [0]:
# Text-generation loop

import random
import sys

for epoch in range(1,61):
  print('\nepoch : ', epoch)
  model.fit(x, y, epochs=1, batch_size=128)
  
  start_index = random.randint(0, len(text)-maxlen-1)
  generated_text_org = text[start_index:start_index+maxlen]
  
  print('\n---Generating with seed : "' + generated_text_org + '"')
  
  for temperature in [0.2, 0.5, 1.0, 1.2]:
    generated_text = generated_text_org
    print('\n---Temperature : ', temperature)
    sys.stdout.write(generated_text)
    
    for i in range(400):
      sampled = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1
    
      preds = model.predict(sampled, verbose=0)[0]
      next_index = sample(preds, temperature)
      next_char = chars[next_index]
    
      generated_text += next_char
      generated_text = generated_text[1:]
    
      sys.stdout.write(next_char)



epoch :  1
Epoch 1/1
200278/200278 [==============================] - 197s 981us/step - loss: 1.6334

---Generating with seed : "s "for the glory of god," or "for the good of the
soul":--th"

---Temperature :  0.2
s "for the glory of god," or "for the good of the
soul":--the such a previsely the such the self and so the such a soul of the present and all the such all the self all the self all the soul only and something and the self and so the self and and conscience of the the seriming of the self all the self and man it is a something the seristhen of the serile of the self the self the such a something that it is all the seriming in the consection of the seriment
---Temperature :  0.5
s "for the glory of god," or "for the good of the
soul":--the self and so the soul is not only and mast and a idea on which conscious and his preasen of such a sence entimate of the said and prises and assuced that it would has all instinct, the dangerous ever the man indifferent not the suprepoing th

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


h feels still
at riling gladetainle.

, something forthi; urmin--dread that, as "feel means eshorable
effection
epoch :  8
Epoch 1/1
200278/200278 [==============================] - 219s 1ms/step - loss: 1.4039

---Generating with seed : "penhauer's teaching
that by it our feelings are temporarily "

---Temperature :  0.2
penhauer's teaching
that by it our feelings are temporarily and and in the sense of the spirit to the soul to be a soul is also the sense of the sense of the senses of the all the sense of the sense of the sense of the other and the sense of the senses of the sense of the sense of the states of the most soul is the sense of the sense of the soul the same the sense of the sense of the spirit of the sense of the states and self-desisting, and the sense of th
---Temperature :  0.5
penhauer's teaching
that by it our feelings are temporarily as some present and the word of such a soul, and why develops, and all the most man are so the
spirit in the same problem not the sen